In [1]:
import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
import utils
%matplotlib inline
import sys
sys.path.append ('../../../../src/nn') 
from costs import NN_CostFunction

In [2]:
# 20x20 Input Images of Digits
input_layer_size  = 400

# 10 labels, from 1 to 10 (note that we have mapped "0" to label 10)
num_labels = 10

#  training data stored in arrays X, y
data = loadmat(os.path.join('Data', 'ex4data1.mat'))
X, y = data['X'], data['y'].ravel()

# set the zero digit to 0, rather than its mapped 10 in this dataset
# This is an artifact due to the fact that this dataset was used in 
# MATLAB where there is no index 0
y[y == 10] = 0

m = y.size
X = np.insert (X,0,1,axis=1)
print (X.shape)

(5000, 401)


In [15]:
# Setup the parameters you will use for this exercise
input_layer_size  = 400  # 20x20 Input Images of Digits
hidden_layer_size = 25   # 25 hidden units
num_labels = 10          # 10 labels, from 0 to 9

# Load the .mat file, which returns a dictionary 
#weights = loadmat(os.path.join('Data', 'ex4weights.mat'))
paramFile = 'data/ex4weights.mat'
params = scipy.io.loadmat(paramFile)
Theta1 = params['Theta1']
Theta2 = params['Theta2']


# get the model weights from the dictionary
# Theta1 has size 25 x 401
# Theta2 has size 10 x 26
#Theta1, Theta2 = weights['Theta1'], weights['Theta2']

# swap first and last columns of Theta2, due to legacy from MATLAB indexing, 
# since the weight file ex3weights.mat was saved based on MATLAB indexing
Theta2 = np.roll(Theta2, 1, axis=0)


NameError: name 'scipy' is not defined

In [4]:
print(Theta1.shape)
print(Theta2.shape)

(25, 401)
(10, 26)


In [16]:

n_layers = 2
input_dimension = 400
nodes_in_layer = [input_dimension,25,10]

# Inicializo Theta para cada layer

Thetas = [Theta1,Theta2]
"""
for i in range (1,n_layers+1):
    theta = np.zeros ((nodes_in_layer[i],nodes_in_layer[i-1])) # Inizializo weigths
    theta = np.insert (theta,0,1,axis=1)  # Inserto BIAS
    Thetas.append (theta)
Thetas = np.array (Thetas)
"""
def sigmoid (Z):
    return 1 / (1 + np.exp (-Z))



def forward_prop (X,Theta):
    ai = X 
    cache = {}
    for i in range(len (Theta)):
        zi = np.dot (ai , Theta [i].T )
        ai = sigmoid (zi)
        cache ['z'+str(i+1)] = zi
        if i != len (Theta)-1:
            ai = np.insert (ai,0,1, axis=1)
        cache ['a'+str(i+1)] = ai

    return ai , cache


def backward_prop (X,Y,Thetas):
    Delta = []
    for i in range (len(Thetas)):
        Delta.append ( np.zeros (Thetas[i].shape))
    prediction,cache = forward_prop (X,Thetas)
    prediction = np.array (prediction)
    print ('Prediction shape',prediction.shape)
    print ('Y shape' , Y.shape)
    delta = {}
    deltai = np.sum(prediction - Y,axis=0)
    ddd = prediction - Y
    print ('dddd',ddd.shape)
    delta ['delta'+str(len (Delta)) ] = deltai 
    for i in reversed (range(1,len(Delta))):
        print (i)
        ai = cache['a'+str(i)]
        gprima_z = ai * ( 1 - ai )
        print ('gprima_z shape',gprima_z.shape)
        print ('deltai shape',deltai.shape)
        print ('thetai shape',Thetas[i].shape)
        step1 = np.dot(Thetas[i].T,deltai.T)
        deltaiprev = (step1.T * gprima_z).sum(axis=0)
        delta ['delta'+str(i) ] = deltaiprev 
        print ('deltaiPREV shape',deltaiprev.shape)
        deltai = deltaiprev
        
    for i in range (1,len(Delta)+1):
        step2 = np.dot (delta['delta'+str(i)],cache['a'+str(i)].T)
        print (step2.shape)
        print (Delta[i-1].shape)
        print ('cachei shape ', cache['a'+str(i)].shape)
        print ('deltai shape ', delta['delta'+str(i)].shape)
        Delta [i-1] = Delta [i-1] + np.dot (cache['a'+str(i)], delta['delta'+str(i)])

    assert J.shape == ()

    return J

        
def NNcostFunction ( A_last, Y , Thetas, l2_lambda=0.0 ):
    cost = NN_CostFunction (A_last, Y )
    if l2_lambda != 0.0:
        # Calculamos regularization
        m = Y.shape [0]  # Número de muestras
        l2_cost = (l2_lambda / ( 2 * m ) ) * reduce (lambda ws, w: ws + np.sum(np.square(w)),Thetas,0)
        cost += l2_cost
        print ('costFunction, regularization value {} , with lambda {}'.format(l2_cost,l2_lambda))
    return cost


In [6]:
labels = y
ysaved = np.copy (y)
y = np.zeros((m,num_labels))
for i in range(m):
    y[i, labels[i]-1] = 1
print (y.shape)
print (y[0:5])


(5000, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [17]:
from functools import reduce
print ('Theta1',Thetas[0][0:1])
print ('Theta2',Thetas[1][0:1])
prediction,cache = forward_prop (X,Thetas)
print (prediction[0:10])
pred = np.argmax (prediction,axis=1)
print('Training Set Accuracy: {:.2f}%'.format(np.mean(pred == ysaved) * 100))
cost = NNcostFunction (prediction, y,Thetas,l2_lambda=0.0)
print (cost)
#backward_prop (X,Y,np.array(Thetas))

Theta1 [[-2.25623899e-02 -1.05624163e-08  2.19414684e-09 -6.92223359e-06
   8.88278871e-05 -2.42735628e-04 -2.70957302e-03  1.45757191e-03
   2.46026364e-03  7.13629532e-04 -5.70078885e-04 -2.31464986e-04
   5.12911249e-04  1.13402876e-03  3.62593523e-04  4.12173828e-04
   1.32134340e-04  1.37730345e-04  1.07593046e-05 -2.12161426e-06
   1.14186154e-09  2.20317337e-08 -4.95776468e-07  7.96752261e-05
  -7.40617279e-04 -4.30419304e-04  2.15555175e-03  9.27076318e-03
   1.50701482e-02  7.36894310e-03  7.66312246e-03  6.38584723e-04
  -8.66664392e-03 -1.36946721e-02 -1.80244907e-03 -1.98550526e-03
  -1.33854878e-03  4.11799777e-05  3.61518629e-03  3.27367511e-04
  -1.02609244e-04 -2.51764893e-07  1.72926132e-05  4.30306233e-04
  -4.31866730e-03 -2.44277714e-02 -4.29677743e-02 -2.34169922e-02
   4.87858916e-02  1.25561218e-01  7.77852907e-02  3.98915622e-02
   3.34552027e-02 -2.16309907e-02 -9.33166492e-02 -9.14928745e-02
  -2.39920603e-03 -2.56924069e-02 -7.66599683e-02 -3.37715197e-03
   

In [32]:
print (prediction.shape)

(5000, 10)


2
1
